# Алгоритмы на потоках данных

На этом семинаре посмотрим на то, что можно сделать с большими данными, когда наши вычислительные возможности несколько ограничены. 

Ограниченость может быть вызвана различными факторами
* У нас просто нет вычислительных ресурсов для hadoop кластера - есть только одна тачка для вычислений с жесткий диском
* У нас все таки есть хадуп кластер, но при этом некоторые задачи все равно решаются на нем мучительно долго
* У нас есть хадуп кластер, однако данные в огромных количествах прилетают каждую секунду (например сообщения из кафки или логи веб-серверов)
* У нас есть только "умная" кофеварка и тонны данных для анализа - например мы строим Internet-of-things и хотим встроить какую-то аналитику в систему. В таком раскладе у нас есть небольшое устройство, которое подключено в гигантской сети из таких же устройств, каждое из которых непрерывно шлет показания с датчиков.

Далеко не любую задачу можно решить за приемлимое время с приемлимым качеством в таких условиях, однако существует целый класс алгоритмов, которые умеют выдавать разумные результаты в подобых сценариях - стриминговые алгоритмы.

Основные отличия стриминговых алгоритмов следующие:
* Память у алгоритма ограничена и много меньше размера входных данных
* Алгоритм может посмотреть на данные только 1 раз

## Датасет

Датасет на сегодняшний семинар - данные с сенсоров "умного" города в Денмарке. Сенсоры снимают показания о разруженности дорог по городу и сливают их в единый поток данных.

Информация по датасету - http://iot.ee.surrey.ac.uk:8080/datasets.html#traffic


In [1]:
! wget http://iot.ee.surrey.ac.uk:8080/datasets/traffic/traffic_oct_nov/citypulse_traffic_raw_data_aarhus_oct_nov_2014.zip 

--2022-03-26 12:06:16--  http://iot.ee.surrey.ac.uk:8080/datasets/traffic/traffic_oct_nov/citypulse_traffic_raw_data_aarhus_oct_nov_2014.zip
Resolving iot.ee.surrey.ac.uk (iot.ee.surrey.ac.uk)... 131.227.92.114
Connecting to iot.ee.surrey.ac.uk (iot.ee.surrey.ac.uk)|131.227.92.114|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38520470 (37M) [application/zip]
Saving to: ‘citypulse_traffic_raw_data_aarhus_oct_nov_2014.zip’

citypulse_traffic_r 100%[===================>]  36.74M  33.7MB/s    in 1.1s    

2022-03-26 12:06:17 (33.7 MB/s) - ‘citypulse_traffic_raw_data_aarhus_oct_nov_2014.zip’ saved [38520470/38520470]



In [2]:
! unzip citypulse_traffic_raw_data_aarhus_oct_nov_2014.zip -d dataset/

Archive:  citypulse_traffic_raw_data_aarhus_oct_nov_2014.zip
  inflating: dataset/trafficData158324.csv  
  inflating: dataset/trafficData158355.csv  
  inflating: dataset/trafficData158386.csv  
  inflating: dataset/trafficData158415.csv  
  inflating: dataset/trafficData158446.csv  
  inflating: dataset/trafficData158475.csv  
  inflating: dataset/trafficData158505.csv  
  inflating: dataset/trafficData158536.csv  
  inflating: dataset/trafficData158565.csv  
  inflating: dataset/trafficData158595.csv  
  inflating: dataset/trafficData158624.csv  
  inflating: dataset/trafficData158655.csv  
  inflating: dataset/trafficData158684.csv  
  inflating: dataset/trafficData158715.csv  
  inflating: dataset/trafficData158744.csv  
  inflating: dataset/trafficData158776.csv  
  inflating: dataset/trafficData158805.csv  
  inflating: dataset/trafficData158836.csv  
  inflating: dataset/trafficData158865.csv  
  inflating: dataset/trafficData158895.csv  
  inflating: dataset/trafficData158924.

In [3]:
! sed -i '1d' dataset/*

In [4]:
! cat dataset/* > traffic.csv

In [5]:
! cat traffic.csv | wc -l

4382599


In [6]:
! head traffic.csv

Колонки датасета следующие
```
status	avgMeasuredTime	avgSpeed	extID	medianMeasuredTime	TIMESTAMP	vehicleCount	_id	REPORT_ID
```

## Простые статистики

### Среднее

Одним из самых простых стриминговых алгоритмов, который писал скорее всего каждый - это подсчет среднего значения набора чисел. 

In [7]:
%%writefile mean-stream.py

import sys
import csv

stream = map(lambda x: int(x[6]), csv.reader(iter(sys.stdin.readline, '')))

vehicle_count = 0
record_count = 0

for current_vehicle_count in stream:
    vehicle_count += current_vehicle_count
    record_count += 1

print(vehicle_count / record_count)

Writing mean-stream.py


Видно, что алгоритм использует O(1) памяти (бкувально две переменные) и проходится по всем данных ровно один раз. 
Это практически эталонный пример того, как структурно выглядит стриминговый алгоритм и далее мы будем говорить именно о подобных алгоритмах.

In [8]:
! cat traffic.csv | tqdm --total 4382599 | python3 mean-stream.py

100%|█████████████████████████████| 4382599/4382599 [00:08<00:00, 520125.65it/s]
3.0282777411303203


Можно заметить, что мы решили эту задачу точно (не приближенно). Аналогично мы можем посчитать и другие несложные статистики - количество, минимум, максимум, дисперсию и так далее.

Однако не все статистики считаются с такой легкостью. Например есть большие проблемы с подсчетом медианы в один проход. 

## Сложные статистики

### Медиана

Ниже предлагается к ознакомлению алгоритм для поиска медианы (Мунро-Патерсон).

Идея крайне простая - возьмем T первых элементов из потока. Далее для всех следующих элементов будем подсчитывать, сколько из этих элементов больше (по значению), чем элементы из нашего множества и сколько элементов меньше. Если в конце окажется так, что и тех и тех (больших и меньших) элементов меньше, чем половина всех элементов в потоке (< N/2), то это означает, что наше множество содержит элементы как раз из середины упорядоченного ряда. А раз так, значит медиана - один из элементов нашего множества. Достаточно будет отсортировать наше множество и взять соответствующий элемент.

Важно отметить, что будут элементы, которые не будут больше или меньше всех элементов нашего множества - они будут где-то между. В этот момент мы просто включим этот элемент в наше множество. Однако так как память у нас ограничена, то мы должны выкинуть какой-то элемент из множества, чтобы расход памяти не увеличивался. Легко заметить, что мы можем избавиться от минимального или максимального элемента нашего множества - если выкидываем максимум, то просто говорим, что на 1 увеличилось число элементов, больших чем наше (симметрично с минимумом). 

Осталось решить что выкинуть - минимум или максимум. Так как в конце мы бы хотели, чтобы больших и меньших элементов было примерно поровну, то тогда будем выкидывать элемент в соответствии с этим желанием - если меньших меньше, то выкидываем минимум, если больших - максимум.

Более формальное описание алгоритма смотри здесь - https://www.cs.dartmouth.edu/~ac/Teach/data-streams-lecnotes.pdf

Ниже - упрощенный схематичный пример работы алгоритма.

![Munro-Paterson](https://github.com/ADKosm/lsml-seminars-2020-public/blob/master/img/munro-paterson.png?raw=true)

In [9]:
%%writefile median-stream.py

import sys
import csv
from itertools import chain

MEMORY = 5000
A = {}

def add(element, number):
    A[element] = A.get(element, 0) + number
    if A[element] == 0:
        A.pop(element)


if len(sys.argv) > 1:
    col = int(sys.argv[1])
else:
    col = 6
    
stream = map(lambda x: int(x[col]), csv.reader(iter(sys.stdin.readline, '')))

for _ in range(MEMORY):
    add(next(stream), 1)

A_min = min(A)
A_max = max(A)

larger = 0
less = 0
N = len(A)

for element in stream:
    N += 1
    if element > A_max:
        larger += 1
    elif element < A_min:
        less += 1
    else:
        if less < larger:
            add(A_min, -1)
            add(element, +1)
            A_min = min(A)
            less += 1
        else:
            add(A_max, -1)
            add(element, +1)
            A_max = max(A)
            larger += 1

if less < N / 2 and larger < N / 2:
    median_index = N // 2 - less
#     A = sorted(list(A))
    A = list(chain.from_iterable([[value] * count for value, count in A.items()]))
    result = A[median_index]
    print(result)
else:
    print("FAIL")
    print(N, less, larger, A_min, A_max)

Writing median-stream.py


In [10]:
! cat traffic.csv | tqdm --total 4382599 | python3 median-stream.py 6

100%|█████████████████████████████| 4382599/4382599 [00:09<00:00, 481414.07it/s]
FAIL
4377630 3051379 1326220 3 3


In [11]:
! cat traffic.csv | shuf | tqdm --total 4382599 | python3 median-stream.py 6

100%|█████████████████████████████| 4382599/4382599 [00:10<00:00, 418702.63it/s]
1


In [12]:
! cat traffic.csv | shuf | tqdm --total 4382599 | python3 median-stream.py 6

100%|█████████████████████████████| 4382599/4382599 [00:10<00:00, 417793.46it/s]
1


In [13]:
! cat traffic.csv | tqdm --total 4382599 | python3 median-stream.py 1

100%|█████████████████████████████| 4382599/4382599 [00:08<00:00, 497303.89it/s]
FAIL
4377748 1560787 2816812 64 64


In [14]:
! cat traffic.csv | shuf | tqdm --total 4382599 | python3 median-stream.py 1

100%|█████████████████████████████| 4382599/4382599 [00:10<00:00, 435877.34it/s]
80


Можно заметить, что алгоритм работает только на потоках, которые хорошо перемешаны. Если же нам с данными не повезет, то все пропало.

Можно ли находить медиану всегда с ограничением по памяти? Кажется, что эта задача близка к невыполнимой. Есть продвинутые алгоримты, которые всегда выдают ответ, однако их результат - лишь статистическая оценка, а не точный ответ. 

Например алгоритм **P-Square**. Он позволяет оценивать любые квантили на потоке. Подробнее можно почитать про него в оригинальной статье - https://www.cse.wustl.edu/~jain/papers/ftp/psqr.pdf . Сейчас останавливаться на нем мы не будем.

## Скетчи

### Принадлежность множеству

Помимо подсчета каких-то статистик, часто возникает задача построить структуру данных, которая бы смогла отвечать нам на какие-то запросы после обработки потока.

Например - **присутствовал ли такой элемент в потоке.**

Если бы у нас было O(N) памяти, то тогда эту задачу можно было бы решить честно. Однако нам такой расклад не подходит, поэтому будем строить алгоритм, который отвечает на вопрос правильно с некоторой информацией, но при этом используя гораздо меньше памяти.

Здесь нам могут помочь хеш-функции. Самая простая идея, которая нам может прийти в голову - запоминать не сами увиденные значения, а их хеши.

План действий следующий: заведем массив размера T - здесь будем отмечать элементы, которые мы видели.
Будем хешировать все элементы, которые есть в потоке, в отрезок [0, T] (T выбирается исходя из размера доступной памяти) и отмечать соответствующий элемент в нашем массиве как увиденный. После того, как мы обработаем таким образом весь массив мы можем обабатывать входящие запросы.

Для входящего запроса посчитаем хеш элемента, который нас спросили и проверим, есть ли он у нас в массиве.
Если в массиве указано, что такой хеш мы не видели, значит и сам элемент мы точно не видели - ответ нет.
Если же указано, что видели - тогда возможно, что такой элемент присутствовал в потоке, а может и нет. Такое может произойти, когда хеш другого элемента из потока совпал с хешом элемента, про который спросили. В данной ситуации мы отвечаем, что видели, однако нужно держать в голове, что этот ответ может быть неверным с некоторой вероятностью.

Для того, чтобы уменьшить вероятность ошибки, мы можешь применить следующий трюк - возьмем сразу P различных случайных хеш-функций. Будем отмечать в нашем массиве сразу все значения хешей, как увиденные.

При ответе также возьмем все P хешей от элемента. Если хотя бы один из значений хеша отсутствует в нашем массиве, то значит такой элемент мы не видели.
Если же все хеши присутствуют в массиве, значит или мы видели этот элемент, или нам очень не повезло и у нас случилось сразу P коллизий (что менее вероятно, чем в первом случае в одной хеш функцией).

Структура данных, которую мы только что описали, называется **Bloom Filter**.

Более подробное описание можно посмотреть здесь - https://shodhganga.inflibnet.ac.in/bitstream/10603/11703/9/09_chapter%204.pdf
В статье также можно найти псевдокод работы алгоритма.

Ниже - схема принципа работы струткуры.

![Bloom-filter](https://github.com/ADKosm/lsml-seminars-2020-public/blob/master/img/bloom-filter.png?raw=true)

<sub><sup>Картинка взята из https://en.wikipedia.org/wiki/Bloom_filter</sup></sub>

Может возникнуть справедливый вопрос - где взять много случайных хеш-функций. Самый простой вариант - взять в качестве хеш-функции парамеризуемую функцию и случайно выставлять параметры.

In [15]:
import random

def generate_hash(size):
    p1, p2, p3 = random.randint(10, 10**8), random.randint(10, 10**8), random.randint(10, 10**8)
    
    def _hash(value):
        return (p1 + value * p2 + value ** 2 * p3) % size
    
    return _hash

In [16]:
h1 = generate_hash(20)
h2 = generate_hash(20)

In [17]:
print(h1(10))
print(h2(10))

18
14


In [18]:
print(h1(10))
print(h1(21))

18
6


In [19]:
%%writefile bloom-filter.py
import argparse
import csv
import random
import sys


MEMORY = 5000
HASHES_COUNT = 5


def generate_hash(size, seed=None):
    random.seed(seed)
    p1, p2, p3 = random.randint(10, 10**8), random.randint(10, 10**8), random.randint(10, 10**8)

    def _hash(value):
        return (p1 + value * p2 + value ** 2 * p3) % size

    return _hash


def main(query_file_path):
    stream = map(lambda x: int(x[8]), csv.reader(iter(sys.stdin.readline, '')))

    bit_array = [False]*MEMORY
    hash_funcs = [generate_hash(MEMORY, i) for i in range(HASHES_COUNT)]

    for element in stream:
        for h in hash_funcs:
            bit_array[h(element)] = True

    with open(query_file_path, "r") as f:
        for query in map(int, f):
            if all(bit_array[h(query)] for h in hash_funcs):
                print('YES')
            else:
                print('NO')


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Process some integers.')
    parser.add_argument('queries', metavar='query', type=str, nargs='?',
                        help='path to queries file')
    args = parser.parse_args()
    main(args.queries)

Writing bloom-filter.py


In [20]:
%%writefile bloom-filter-query.txt
158324
203546
158776
23
894
180926
182984
81511

Writing bloom-filter-query.txt


In [21]:
! cat traffic.csv | tqdm --total 4382599 | python3 bloom-filter.py bloom-filter-query.txt

100%|█████████████████████████████| 4382599/4382599 [00:18<00:00, 239549.83it/s]
YES
NO
YES
NO
NO
YES
NO
NO


### Подсчет частоты 

Зададимся немного более сложным вопросом - **сколько раз заданный элемент встречался в потоке.**

Как и с просто проверкой наличия элемента в потоке - эту задачу не получится решать честно в заданных условиях. Ответ опять будет примерный.

Попробуем продолжить идею использования хешей для решения этой задачи. Опять возьмем массив размера T в который будем записывать, сколько раз мы увидели тот или иной хеш. Для каждого нового элемента считаем хеш и увеличиваем соответствующий счетчик в массиве.

Когда нам придет запрос - посчитаем хеш и посмотрим в массив. Число будет скорее всего больше, чем правильный ответ, так как из-за коллизий в соответствующую ячейку добавились результаты от элементов, которые имеют одинаковый хеш.

Для того, чтобы уменьшить масштаб трагедии из-за этих коллизий опять воспользуемся приемом с несколькими хеш-функциями. Возьмем теперь сразу несколько массивов и для каждого массива возьмем свою случайную хеш-функцию.
Для каждого массива будем проделывать такие же операции.

Теперь когда к нам придет запрос - посчитаем всех хеши от элемента и посмотрим во все соответствующие ячейки в массивах. Все эти значения очевидно не меньше чем правильный ответ, а значит минимум из этих чисел - наиболее точная оценка того, сколько на самом деле раз мы видели этот элемент в потоке. Его и дадим в качестве ответа.

Та конструкция, которую мы только что построили, называется **Count Min Sketch**.

Более подробное описание можно посмотреть здесь - http://resources.mpi-inf.mpg.de/departments/d1/teaching/ss13/gitcs/lecture5.pdf.

Ниже - схема принципа работы структуры.

![count-min-sketch](https://raw.githubusercontent.com/ADKosm/lsml-seminars-2020-public/master/img/count-min-sketch.png)

<sub><sup>Картика взята из https://github.com/gopalkrushnapattanaik/SystemDesign/wiki/Count-Min-Sketch</sup></sub>

In [22]:
%%writefile count-min-sketch.py
import argparse
import csv
import random
import sys

TABLE_SIZE = 5000
HASHES_COUNT = 10


def generate_hash(size, seed=None):
    random.seed(seed)
    p1, p2, p3 = random.randint(10, 10**8), random.randint(10, 10**8), random.randint(10, 10**8)

    def _hash(value):
        return (p1 + value * p2 + value ** 2 * p3) % size

    return _hash


def main(query_file_path):
    stream = map(lambda x: int(x[8]), csv.reader(iter(sys.stdin.readline, '')))

    cnt_table = [[0 for _ in range(TABLE_SIZE)] for _ in range(HASHES_COUNT)]
    hash_funcs = [generate_hash(TABLE_SIZE, i) for i in range(HASHES_COUNT)]

    for element in stream:
        for i, h in enumerate(hash_funcs):
            cnt_table[i][h(element)] = cnt_table[i][h(element)] + 1

    with open(query_file_path, "r") as f:
        for query in map(int, f):
            query_cnt = min(cnt_table[i][h(query)] for i, h in enumerate(hash_funcs))
            print(query_cnt)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Process some integers.')
    parser.add_argument('queries', metavar='query', type=str, nargs='?',
                        help='path to queries file')
    args = parser.parse_args()
    main(args.queries)

Writing count-min-sketch.py


In [23]:
%%writefile count-min-sketch-query.txt
158324
203546
158776
23
894
180926
182984
81511
187774
201855
190100

Writing count-min-sketch-query.txt


In [24]:
! cat traffic.csv | tqdm --total 4382599 | python3 count-min-sketch.py count-min-sketch-query.txt

100%|██████████████████████████████| 4382599/4382599 [00:55<00:00, 78897.72it/s]
9305
0
9958
0
0
9919
0
0
9993
9756
9451


### Количество уникальных элементов в потоке

Одной из важный задач является подсчет **количества уникальных элементов в потоке.** В этот раз так просто составить табличку и просто хешировать в нее не получится. 

Самыми продвинутыми алгоритмами в этом классе считаются LogLog алгоритмы (LogLog, HyperLogLog, HLL++ и другие). Про них есть много литературы (например http://algo.inria.fr/flajolet/Publications/FlMa85.pdf ), но они используют зубодробительный тервер и разбирать их здесь мы не будем. Но важно знать, что такие алгоритмы есть и понимать, какую задачу они решают. 

На семинаре предлагается рассмотреть более простой, но все еще работающий способ решения задачи.

Давайте посмотрим на двоичную запись хеша от элемента.

In [28]:
hash_function = generate_hash(10**4)

for number in range(10):
    print(bin(hash_function(number)))

0b1100111
0b10000100101010
0b100101001
0b1010110010100
0b1000001001011
0b1100001011110
0b11010111101
0b1001111000
0b101110001111
0b10001000000010


Давайте посмотрим на количество нулей в конце двоичной записи.
Если хеш случайный, то вероятность того, что на конце будет 1 = 1/2. 
Вероятность того, что на конце будет 10 = 1/4. 100 - 1/8 и так далее.

In [29]:
def zeros(number):
    result = 0
    while number and number & 1 == 0:
        result += 1
        number = number >> 1
    return result

for number in range(10):
    print(bin(hash_function(number)), zeros(hash_function(number)))

0b1100111 0
0b10000100101010 1
0b100101001 0
0b1010110010100 2
0b1000001001011 0
0b1100001011110 1
0b11010111101 0
0b1001111000 3
0b101110001111 0
0b10001000000010 1


Это так же означает, что в множестве элементов примерно половина будет с нулем 0 на конце их хеш-функции, примерно четверть с одним 0 на конце, восьмая часть с двумя 0 на конце и тд. Этот факт нас очень скоро потребуется.

Итак, если бы у нас было неограниченно памяти, мы бы в таком случае просто складывали все элементы из потока в множество и в конце просто бы посмотрели на размер этого множества - это был бы честный ответ в данной задаче.

Учитывая, что память у нас ограничена, нам придется каким-то образом ужимать это множество. План следующий:

* Возмем пустое множество B и отдельный счетчик z = 0
* Для входящих элементов из потока будет добавлять в множество этот элемент.
* Как только мы увидим, что размер множества превзошел определенный порог (лимит по памяти) производим следующую операцию
  * Из множества удаляем все элементы у которых ровно z нулей на конце хеша (в первый раз будет 0 нулей, то есть те, у которых хеш оканчивается на 1)
  * Увеличиваем z на 1
* Далее все следующие элементы добавляем в множество, только если количество нулей на конце хеша не меньше z. 
* Как только в следующий раз у нас опять множество "переполняется", вновь повторяем процедуру с очисткой множества и увеличения z
* В конце необходимо по нашему получившемуся множеству и z восстановить, сколько же различных элементов мы видели

Каждая операция очистки множества удаляла из него примерно половину элементов (это мы увидели выше). Таким образом, исходное количество различных элементов в множестве будет равно примерно |B| * 2^z.

Это и будет нашим ответом.

Описанный алгоритм называется BJKST.

Более подробное описание можно посмотреть здесь - http://resources.mpi-inf.mpg.de/departments/d1/teaching/ss13/gitcs/lecture5.pdf. В статье также можно найти псевдокод работы алгоритма.

Ниже - схема принципа работы струткуры.

![bjkst](https://github.com/ADKosm/lsml-seminars-2020-public/blob/master/img/bjkst.png?raw=true)

In [30]:
%%writefile bjkst.py
import csv
import random
import sys


MEMORY = 100


def generate_hash(size, seed=None):
    random.seed(seed)
    p1, p2, p3 = random.randint(10, 10**8), random.randint(10, 10**8), random.randint(10, 10**8)

    def _hash(value):
        return (p1 + value * p2 + value ** 2 * p3) % size

    return _hash


def zeros(number):
    result = 0
    while number and number & 1 == 0:
        result += 1
        number = number >> 1
    return result


def main():
    stream = map(lambda x: int(x[8]), csv.reader(iter(sys.stdin.readline, '')))

    z = 0
    B = set()

    for element in stream:
        if len(B) > MEMORY:
            B = {b for b in B if zeros(b) == z}
            z += 1

        if zeros(element) >= z:
            B.add(element)

    result = len(B) * 2**z
    print(result)


if __name__ == '__main__':
    main()

Writing bjkst.py


In [31]:
! cat traffic.csv | tqdm --total 4382599 | python3 bjkst.py

100%|█████████████████████████████| 4382599/4382599 [00:09<00:00, 477975.46it/s]
312
